In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from art_skills import SlnStrokeExpression
import scipy.optimize

## Computing SLN stroke speed and distance traveled
$$ \Lambda(t) = \frac{1}{\sigma\sqrt{2\pi}(t-t_0)} \exp{\left(-\frac{(\ln(t-t_0)-\mu)^2}{2\sigma^2}\right)} $$

In [2]:
def evaluate_speed(t, t0, D, th1, th2, sigma, mu):
    t = np.maximum(t, t0 + 1e-9)
    if False:   # use c++ wrapped version
        sln_stroke = SlnStrokeExpression(t0, D, th1, th2, sigma, mu)
        return np.array([sln_stroke.speed(t_) for t_ in t])
    else:       # use numpy version
        return D / (sigma * np.sqrt(2 * np.pi) * (t - t0)) * np.exp(-np.square(np.log(t - t0) - mu) /
                                                                 (2 * sigma * sigma))
def evaluate_position(t, t0, D, th1, th2, sigma, mu):
    return np.hstack((0, np.cumsum(evaluate_speed(t, t0, D, th1, th2, sigma, mu)[1:] * np.diff(t))))

## Plotting SLN strokes to gain intuition about parameters

First make an interactive plot where we can drag sliders to change the SLN stroke parameters

In [3]:
def plot_sln_single(t0, D, th1, th2, sigma, mu):
    if True:  # this tries to cut off the curve at 90% of the probability density
        import scipy.special
        print(scipy.special.erfinv(0.9) * np.sqrt(2))
        tmax = np.exp(mu + np.sqrt(2)*sigma * scipy.special.erfinv(0.9))*2
        t = np.logspace(-5, np.log10(tmax), 100)
    else:
        # tmax = np.exp(1*sigma + mu) + t0
        tmax = 10
        t = np.logspace(-5, 1, 100)
    v = evaluate_speed(t, t0, D, th1, th2, sigma, mu)
    x = evaluate_position(t, t0, D, th1, th2, sigma, mu)

    if D == 1:  # this is just a "unit test check" for computing the extremal point
        tpeak_alt = np.exp(mu - sigma * sigma)
        predicted_peak_speed = 1 / (sigma * np.sqrt(2 * np.pi) *
                                    (tpeak_alt)) * np.exp(-0.5 * sigma * sigma)


    plt.figure(figsize=(12, 4))
    plt.subplot(131)
    plt.plot(np.log(t - t0), v)
    plt.xlim(-2.5, 2.5)
    plt.xlim(-np.log(tmax-t0),np.log(tmax-t0))
    plt.ylim(0, 1)
    plt.title('Gaussian Curve: Speed vs ln(t-t0)')
    plt.subplot(132)
    plt.plot(t, v)
    if D == 1:
        plt.plot(tpeak_alt + t0, predicted_peak_speed, 'r*')
    # plt.ylim(0, 1)
    # plt.xlim(0, 0.6)
    plt.title('Speed vs Time')
    plt.subplot(133)
    plt.plot(t, x)
    # plt.xlim(0, 0.6)
    # plt.ylim(0, 100)
    plt.title('Distance Traveled vs Time')
    plt.show()
interactive_plot = widgets.interactive(plot_sln_single,
                                       t0=widgets.FloatSlider(min=-5, max=-1e-12, value=-0.1),
                                       D=widgets.FloatSlider(min=0, max=5, value=1),
                                       th1=widgets.fixed(0),
                                       th2=widgets.fixed(0),
                                       sigma=widgets.FloatSlider(min=1e-12, max=2, value=1),
                                       mu=widgets.FloatSlider(min=-5, max=5, value=0))

output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


interactive(children=(FloatSlider(value=-0.1, description='t0', max=-1e-12, min=-5.0), FloatSlider(value=1.0, …

We notice that $D$ and $t_0$ are pretty obvious what their effects are, so let's just plot the a bunch of variations of $\sigma, \mu$ in a grid.

Important kwargs are:
* `normalize_x` - re-scales the x-axis so that the peak velocity occurs at 1/15 the way through the plot.
* `normalize_y` - rescales the y-axis to capture 5% past the full range
* `pos` - whether to plot position or velocity

The next 2 cells plot speed and distance traveled, respectively

In [ ]:
def plot_sln_speed_array(ax, t0, D, th1, th2, sigma, mu, normalize_x=True, normalize_y=True):
    if normalize_x:
        duration = 1
        # tmax = -(np.log(1) - 2*sigma)
        tmax = np.exp(np.sqrt(2)*scipy.special.erfinv(0.9) * sigma + mu)
        tmax = np.exp(0.9 + mu)
        t = np.linspace(0, tmax, 100)
        # t_extrema = np.exp(mu - sigma**2) # https://www.wolframalpha.com/input?i=solve+%28derivative+of+1%2Fx+*+exp%28-%28ln%28x%29+-+mu%29%5E2+%2F+%282*sigma%5E2%29%29%29+%3D+0
        # print(t_extrema * 15)
        # t = np.linspace(0, t_extrema * 15, 100)
    else:
        t = np.logspace(-5, 0.3, 100)
    v = evaluate_speed(t, t0, D, th1, th2, sigma, mu)
    ax.plot(t, v)
    if normalize_x:
        ax.set_xlim(t[0], t[-1])
        ax.grid(False)
    else:
        ax.set_xlim(0, 10)
    if normalize_y:
        ax.set_ylim(0, np.max(v)*1.05)
        ax.grid(False)
    else:
        ax.set_ylim(0, 1)

sigma_vals = np.linspace(1e-10, 2, 6)[1:]
mu_vals = np.linspace(-2, 3, 6)
fig, axes = plt.subplots(len(sigma_vals), len(mu_vals), sharex=False, sharey=False, figsize=(20, 14))
for ri, sigma in enumerate(sigma_vals):
    for ci, mu in enumerate(mu_vals):
        plot_sln_speed_array(axes[ri][ci], 0, 1, 0, 0, sigma, mu, normalize_x=True, normalize_y=True)
        axes[0][ci].set_title('$\\mu = {:}$'.format(mu))
    axes[ri][0].set_ylabel('$\\sigma = {:.1f}$'.format(sigma))

In [ ]:
def plot_sln_position_array(ax, sigma, mu):
    t_ = np.linspace(0, 100, 10000)
    x_ = evaluate_position(t_, 0, 1, 0, 0, sigma, 1)
    xmax = x_[-1]
    tmax = np.argmin(np.square(0.8 - x_))

    t = np.linspace(0, t_[tmax]*np.exp(mu), 300)
    x = evaluate_position(t, 0, 1, 0, 0, sigma, mu)
    ax.plot(t, x)
    ax.grid(False)

sigma_vals = np.linspace(1e-10, 2, 5)[1:]
mu_vals = np.linspace(-2, 3, 6)
fig, axes = plt.subplots(len(sigma_vals), len(mu_vals), sharex=False, sharey=False, figsize=(20, 14))
for ri, sigma in enumerate(sigma_vals):
    for ci, mu in enumerate(mu_vals):
        plot_sln_position_array(axes[ri][ci], sigma, mu)
        axes[0][ci].set_title('$\\mu = {:}$'.format(mu))
    axes[ri][0].set_ylabel('$\\sigma = {:.1f}$'.format(sigma))

## Test case for `test_sln_stroke_fit.py`

Generate data for a unit test case

In [4]:
params_gt = np.array([0., 1., 0., 0., 0.5, -1.])
query_times = [0.0, 0.2, 0.3, 0.6]
# Generate ground truth for test_sln_stroke_fit
dt = 0.01
t = np.arange(0, 0.6 + dt, dt)
v = evaluate_speed(t, *params_gt)
x = evaluate_position(t, *params_gt)
# Print out unit test "expected"
t_expected = np.array(query_times).reshape(-1, 1)
x_expected = np.array([x[t == t_expected_][0] for t_expected_ in t_expected]).reshape(-1, 1)
data_expected = np.hstack((t_expected, x_expected, np.zeros(t_expected.shape)))
print('pExpected = np.array(', params_gt.tolist(), ')')
print('data = np.array(', data_expected.tolist(), ')')

pExpected = np.array( [0.0, 1.0, 0.0, 0.0, 0.5, -1.0] )
data = np.array( [[0.0, 0.0, 0.0], [0.2, 0.1210495445244642, 0.0], [0.3, 0.3538806027542822, 0.0], [0.6, 0.8401353767843258, 0.0]] )


Now let's compare the optimization output of the unit test

In [5]:
params_optim_result = [
    0.017215989124497455, 1.0232191058557842, 0.0, 0.0, 0.542680180326043, -1.0303325390125773
]

def print_info(description, *params):
    t = np.arange(0, 0.61, 0.01)
    v = evaluate_speed(t, *params)
    x = evaluate_position(t, *params)
    print('*** {:} ***'.format(description))
    print('parameters:', tuple(params))
    print('first 5 velocities:', v[:5])
    print('first 5 positions:', np.cumsum(v[:5] * 0.01) - v[0]*0.01)
    print('data velocities:', [v[t == t_][0] for t_ in query_times])
    print('data positions:', [x[t == t_][0] for t_ in query_times])
print_info('Ground Truth', *params_gt)
print_info('Optimization Result', *params_optim_result)

*** Ground Truth ***
parameters: (0.0, 1.0, 0.0, 0.0, 0.5, -1.0)
first 5 velocities: [0.00000000e+00 4.09892860e-10 1.71941866e-06 9.28153576e-05
 1.05550561e-03]
first 5 positions: [0.00000000e+00 4.09892860e-12 1.71982855e-08 9.45351861e-07
 1.15004079e-05]
data velocities: [0.0, 1.8980317374399713, 2.4472663885848656, 0.824029565210426]
data positions: [0.0, 0.1210495445244642, 0.3538806027542822, 0.8401353767843258]
*** Optimization Result ***
parameters: (0.017215989124497455, 1.0232191058557842, 0.0, 0.0, 0.542680180326043, -1.0303325390125773)
first 5 velocities: [8.47686781e-278 8.47686781e-278 1.15446979e-015 3.95546459e-007
 8.65081496e-005]
first 5 positions: [0.00000000e+000 8.47686781e-280 1.15446979e-017 3.95546460e-009
 8.69036961e-007]
data velocities: [8.476867805356991e-278, 1.9243186842855202, 2.4262753399397363, 0.8580401547713405]
data positions: [0.0, 0.12104954452446418, 0.3538806027542822, 0.8401353767843261]


As we can see, even though the first 5 positions/speeds are a little bit different, the positions/speeds at the 4 data points are virtually perfect matches.

Let's also plot this to see visually how close of a match it is.

In [6]:
# Visualize Unit Test Results
def plot_sln_test(t0, D, th1, th2, sigma, mu, ls='k-'):
    t = np.linspace(0, 0.6, 400)
    v = evaluate_speed(t, t0, D, th1, th2, sigma, mu)
    plt.subplot(131)
    plt.plot(np.log(t - t0), v, ls)
    plt.xlim(-2.5, 2.5)
    plt.title('Gaussian Curve: Velocity vs ln(t-t0)')
    plt.subplot(132)
    plt.plot(t, v, ls)
    plt.xlim(0, 0.6)
    plt.title('Velocity vs Time')
    plt.subplot(133)
    plt.plot(t, np.cumsum(v) - v[0], ls)
    plt.xlim(0, 0.6)
    plt.title('Position vs Time')
plt.figure(figsize=(12, 4))
plot_sln_test(*params, 'k-')
plot_sln_test(*params_optim_result, 'r:')
plt.legend((str(params), str([round(p_, 2) for p_ in p])))
plt.show()

NameError: name 'params' is not defined

<Figure size 1200x400 with 0 Axes>

Even though the underlying "normal" curve is perceptibly different and the lognormal curve (speed vs time) is a teensy bit different, the position vs time is a perfect match as far as our eyes can tell.

## Effect of shape parameter assuming other parameters "normalize"
Let's try computing the optimal values of the parameters $t_0, D, \mu$ such that the resulting curve is most similar to a reference curve, for various values of $\sigma$.  This will enable us to see the effect of changing $\sigma$ only.

In [ ]:
t = np.linspace(0, 1.5, 100)
# t = np.linspace(0, 0.6, 100)

# reference curve
params_ref = 0, 1, 0, 0, 1.5, -2.0
# params_ref = -0.8, 1, 0, 0, 0.2, 0.1
# params_ref = params
v_ref, x_ref = evaluate_speed(t, *params_ref), evaluate_position(t, *params_ref)

# optimization functions
def loss(sigma, x):
    t0, D, mu = x
    return evaluate_position(t, t0, D, 0, 0, sigma, mu) - x_ref
def optim_params(sigma):
    sol, success = scipy.optimize.leastsq(lambda x: loss(sigma, x), (params_ref[0], params_ref[1], params_ref[-1]))
    assert success, 'Optimization failed'
    return sol[0], sol[1], 0, 0, sigma, sol[2]
    # return sol[0], sol[1], 0, 0, sol[2], sigma

# Plot
N = 8
fig, axes = plt.subplots(2, 1, figsize=(6, 9))
axes[0].plot(t, v_ref, 'k-', label='$\\sigma={:}$'.format(params_ref[-2]))
axes[1].plot(t, x_ref, 'k-', label='$\\sigma={:}$'.format(params_ref[-2]))
cmap = plt.cm.get_cmap('autumn')
axes[0].set_prop_cycle(color=[cmap(i) for i in np.linspace(0, 1, N, endpoint=True)])
axes[1].set_prop_cycle(color=[cmap(i) for i in np.linspace(0, 1, N, endpoint=True)])
# for sigma in np.linspace(-1.2, -0.8, N + 1)[1:-3]:
for sigma in np.linspace(0., 3., N + 1)[1:]:
    params = optim_params(sigma)
    v, x = evaluate_speed(t, *params), evaluate_position(t, *params)
    np.set_printoptions(suppress=True)
    params_str = np.array2string(np.array(params), precision=2, separator=',')
    axes[0].plot(t, v, label='$\\sigma={:}$, {:}'.format(round(sigma, 1), params_str), linewidth=1)
    axes[1].plot(t, x, label='$\\sigma={:}$, {:}'.format(round(sigma, 1), params_str), linewidth=1)
axes[0].legend()
axes[1].set_xlabel('$t$', size=24)
axes[1].set_ylabel('$x$', size=24)
axes[0].set_ylabel('$v$', size=24)
axes[0].set_title('Effect of changing $\\sigma$', size=24);